# Regression Exercise 

California Housing Data

This data set contains information about all the block groups in California from the 1990 Census. In this sample a block group on average includes 1425.5 individuals living in a geographically compact area. 

The task is to aproximate the median house value of each block from the values of the rest of the variables. 

 It has been obtained from the LIACC repository. The original page where the data set can be found is: http://www.liaad.up.pt/~ltorgo/Regression/DataSets.html.
 

The Features:
 
* housingMedianAge: continuous. 
* totalRooms: continuous. 
* totalBedrooms: continuous. 
* population: continuous. 
* households: continuous. 
* medianIncome: continuous. 
* medianHouseValue: continuous. 

## The Data

** Import the cal_housing_clean.csv file with pandas. Separate it into a training (70%) and testing set(30%).**

In [1]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import tensorflow as tf
import math

In [2]:
housing = pd.read_csv('/home/pacho/Documentos/Elearning/Udemy/Tensorflow-Bootcamp-master/02-TensorFlow-Basics/cal_housing_clean.csv')

In [3]:
housing.head()

,housingMedianAge,totalRooms,totalBedrooms,population,households,medianIncome,medianHouseValue
0,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0
1,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0
2,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0
3,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0
4,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0


In [4]:
housing.columns

Index(['housingMedianAge', 'totalRooms', 'totalBedrooms', 'population',
       'households', 'medianIncome', 'medianHouseValue'],
      dtype='object')

In [5]:
housing.describe().T

,count,mean,std,min,25%,50%,75%,max
housingMedianAge,20640.0,28.639486,12.585558,1.0000,18.0000,29.0000,37.00000,52.0000
totalRooms,20640.0,2635.763081,2181.615252,2.0000,1447.7500,2127.0000,3148.00000,39320.0000
totalBedrooms,20640.0,537.898014,421.247906,1.0000,295.0000,435.0000,647.00000,6445.0000
population,20640.0,1425.476744,1132.462122,3.0000,787.0000,1166.0000,1725.00000,35682.0000
households,20640.0,499.539680,382.329753,1.0000,280.0000,409.0000,605.00000,6082.0000
medianIncome,20640.0,3.870671,1.899822,0.4999,2.5634,3.5348,4.74325,15.0001
medianHouseValue,20640.0,206855.816909,115395.615874,14999.0000,119600.0000,179700.0000,264725.00000,500001.0000


In [6]:
x_data = housing[['housingMedianAge', 'totalRooms', 'totalBedrooms', 'population',
       'households', 'medianIncome']]
y_data = housing['medianHouseValue']

In [7]:
X_train, X_test, y_train, y_test = train_test_split(x_data,y_data,test_size=0.3, random_state = 101)
index_x_train = X_train.index

### Scale the Feature Data

** Use sklearn preprocessing to create a MinMaxScaler for the feature data. Fit this scaler only to the training data. Then use it to transform X_test and X_train. Then use the scaled X_test and X_train along with pd.Dataframe to re-create two dataframes of scaled data.**

In [8]:
scaler = MinMaxScaler(copy=True, feature_range=(0, 1))

In [9]:
scaler.fit(X_train)

MinMaxScaler(copy=True, feature_range=(0, 1))

In [10]:
X_train = pd.DataFrame(scaler.transform(X_train), columns=X_train.columns)
X_train.index = index_x_train

In [11]:
X_test = pd.DataFrame(scaler.transform(X_test), columns=X_train.columns)

### Create Feature Columns

** Create the necessary tf.feature_column objects for the estimator. They should all be trated as continuous numeric_columns. **

In [12]:
housing.columns

Index(['housingMedianAge', 'totalRooms', 'totalBedrooms', 'population',
       'households', 'medianIncome', 'medianHouseValue'],
      dtype='object')

In [13]:
housing.columns.values[:-1]

array(['housingMedianAge', 'totalRooms', 'totalBedrooms', 'population',
       'households', 'medianIncome'], dtype=object)

In [14]:
feat_cols = [tf.feature_column.numeric_column(x) for x in housing.columns.values[:-1]]

** Create the input function for the estimator object. (play around with batch_size and num_epochs)**

In [15]:
input_func = tf.estimator.inputs.pandas_input_fn(x=X_train, y=y_train, batch_size=10, num_epochs=1000, shuffle=True)

** Create the estimator model. Use a DNNRegressor. Play around with the hidden units! **

In [16]:
dnn_model = tf.estimator.DNNRegressor(hidden_units=[10,10,10],feature_columns=feat_cols)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7feb1e5fc2e8>, '_task_id': 0, '_save_checkpoints_secs': 600, '_save_checkpoints_steps': None, '_task_type': 'worker', '_is_chief': True, '_keep_checkpoint_max': 5, '_num_worker_replicas': 1, '_service': None, '_model_dir': '/tmp/tmpvpeu1er2', '_session_config': None, '_save_summary_steps': 100, '_tf_random_seed': None, '_master': '', '_num_ps_replicas': 0, '_log_step_count_steps': 100, '_keep_checkpoint_every_n_hours': 10000}


##### ** Train the model for ~1,000 steps. (Later come back to this and train it for more and check for improvement) **

In [22]:
dnn_model.train(input_fn=input_func,steps=10000)

INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Restoring parameters from /tmp/tmpvpeu1er2/model.ckpt-10000
INFO:tensorflow:Saving checkpoints for 10001 into /tmp/tmpvpeu1er2/model.ckpt.
INFO:tensorflow:loss = 1.07281e+11, step = 10001
INFO:tensorflow:global_step/sec: 720.842
INFO:tensorflow:loss = 8.28651e+10, step = 10101 (0.145 sec)
INFO:tensorflow:global_step/sec: 902.255
INFO:tensorflow:loss = 4.4921e+10, step = 10201 (0.107 sec)
INFO:tensorflow:global_step/sec: 830.83
INFO:tensorflow:loss = 1.21256e+11, step = 10301 (0.120 sec)
INFO:tensorflow:global_step/sec: 895.769
INFO:tensorflow:loss = 1.1754e+11, step = 10401 (0.112 sec)
INFO:tensorflow:global_step/sec: 854.554
INFO:tensorflow:loss = 7.13637e+10, step = 10501 (0.116 sec)
INFO:tensorflow:global_step/sec: 862.827
INFO:tensorflow:loss = 1.25413e+11, step = 10601 (0.116 sec)
INFO:tensorflow:global_step/sec: 859.388
INFO:tensorflow:loss = 1.88404e+11, step = 10701 (0.116 sec)
INFO:tensorflow:global_step/sec: 792.338


INFO:tensorflow:global_step/sec: 865.649
INFO:tensorflow:loss = 5.08626e+10, step = 17901 (0.115 sec)
INFO:tensorflow:global_step/sec: 875.223
INFO:tensorflow:loss = 5.00312e+10, step = 18001 (0.114 sec)
INFO:tensorflow:global_step/sec: 857.794
INFO:tensorflow:loss = 1.87069e+11, step = 18101 (0.116 sec)
INFO:tensorflow:global_step/sec: 874.012
INFO:tensorflow:loss = 1.50998e+11, step = 18201 (0.116 sec)
INFO:tensorflow:global_step/sec: 869.125
INFO:tensorflow:loss = 4.00568e+10, step = 18301 (0.113 sec)
INFO:tensorflow:global_step/sec: 939.447
INFO:tensorflow:loss = 9.58145e+10, step = 18401 (0.110 sec)
INFO:tensorflow:global_step/sec: 861.774
INFO:tensorflow:loss = 1.41718e+11, step = 18501 (0.115 sec)
INFO:tensorflow:global_step/sec: 856.117
INFO:tensorflow:loss = 6.73572e+10, step = 18601 (0.115 sec)
INFO:tensorflow:global_step/sec: 936.984
INFO:tensorflow:loss = 1.15598e+11, step = 18701 (0.109 sec)
INFO:tensorflow:global_step/sec: 888.411
INFO:tensorflow:loss = 9.28128e+10, step 

** Create a prediction input function and then use the .predict method off your estimator model to create a list or predictions on your test data. **

In [23]:
pred_input_func = tf.estimator.inputs.pandas_input_fn(x=X_test, batch_size=10, num_epochs=1, shuffle=False)

In [24]:
preds = dnn_model.predict(pred_input_func)

In [25]:
predictions = [pred['predictions'] for pred in preds]

INFO:tensorflow:Restoring parameters from /tmp/tmpvpeu1er2/model.ckpt-20000


** Calculate the RMSE. You should be able to get around 100,000 RMSE (remember that this is in the same units as the label.) Do this manually or use [sklearn.metrics](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.mean_squared_error.html) **

In [26]:
math.sqrt(mean_squared_error(y_test, predictions))

97964.60188306686

# Great Job!